In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from matplotlib import cm

In [2]:
radar1 = pd.read_pickle('radar1_s1.pkl')
radar2 = pd.read_pickle('radar2_s1.pkl')

In [3]:
def handle_timesync(df):
    df['datetime'] = df['datetime'].apply(lambda x: pd.to_datetime(x))
    df = df[df.datetime != df['datetime'][0]][df.datetime != df['datetime'][len(df['datetime'])-1]]
    unique_timestamps, counts =  np.unique(df['datetime'], return_counts=True)
    timestamps = []
    delete_row_idx = []
    row_idx = 0
    for unique_timestamp, count in zip (unique_timestamps, counts):
        row_idx +=count
        if count > 5:
            print(f'deleting {row_idx}')
            delete_row_idx.append(row_idx)
            count = 5
        for dele in np.arange(count):
            timestamps.append(pd.to_datetime(unique_timestamp)+(timedelta(milliseconds=200)*dele))
    df = df.drop(df.index[delete_row_idx])
    df['datetime'] = timestamps
    return df

In [4]:
radar1 = handle_timesync(radar1)
radar2 = handle_timesync(radar2)

deleting 109
deleting 119
deleting 144
deleting 244
deleting 254


/tmp/ipykernel_1221912/2502313886.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.datetime != df['datetime'][0]][df.datetime != df['datetime'][len(df['datetime'])-1]]
/tmp/ipykernel_1221912/2502313886.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.datetime != df['datetime'][0]][df.datetime != df['datetime'][len(df['datetime'])-1]]


In [5]:
radar1.columns = [
    f"radar1_{col}" if col != "datetime" else col
    for col in radar1.columns
]
radar2.columns = [
    f"radar2_{col}" if col != "datetime" else col
    for col in radar2.columns
]

In [6]:
radar2['datetime']

2     2025-05-19 21:31:33.000
3     2025-05-19 21:31:33.200
4     2025-05-19 21:31:33.400
5     2025-05-19 21:31:33.600
6     2025-05-19 21:31:33.800
                ...          
181   2025-05-19 21:32:25.000
182   2025-05-19 21:32:25.200
183   2025-05-19 21:32:25.400
184   2025-05-19 21:32:25.600
185   2025-05-19 21:32:25.800
Name: datetime, Length: 259, dtype: datetime64[ns]

In [7]:
mergedRadarData =pd.merge_asof(radar1,radar2, on="datetime", direction="nearest")
mergedRadarData = mergedRadarData.dropna()

In [8]:
mergedRadarData

Index(['radar1_datenow', 'radar1_timenow', 'radar1_rangeIdx',
       'radar1_dopplerIdx', 'radar1_numDetectedObj', 'radar1_x_coord',
       'radar1_y_coord', 'radar1_z_coord', 'radar1_rp_y', 'radar1_noiserp_y',
       'radar1_doppz', 'radar1_interFrameProcessingTime',
       'radar1_interFrameProcessingMargin',
       'radar1_interChirpProcessingMargin', 'radar1_transmitOutputTime',
       'radar1_activeFrameCPULoad', 'radar1_interFrameCPULoad', 'radar1_snrDB',
       'radar1_noiseDB', 'radar1_activity', 'datetime', 'radar2_datenow',
       'radar2_timenow', 'radar2_rangeIdx', 'radar2_dopplerIdx',
       'radar2_numDetectedObj', 'radar2_x_coord', 'radar2_y_coord',
       'radar2_z_coord', 'radar2_rp_y', 'radar2_noiserp_y', 'radar2_doppz',
       'radar2_interFrameProcessingTime', 'radar2_interFrameProcessingMargin',
       'radar2_interChirpProcessingMargin', 'radar2_transmitOutputTime',
       'radar2_activeFrameCPULoad', 'radar2_interFrameCPULoad', 'radar2_snrDB',
       'radar2_nois

In [27]:
x_pcd = [e for elm in mergedRadarData['radar1_x_coord'][0:5] for e in elm]
y_pcd = [e for elm in mergedRadarData['radar1_y_coord'][0:5] for e in elm]
dop_pcd = [e for elm in mergedRadarData['radar1_dopplerIdx'][0:5] for e in elm]

points = np.concatenate((np.array(x_pcd).reshape(-1,1),np.array(y_pcd).reshape(-1,1), np.array(dop_pcd).reshape(-1,1)), axis=1)
points = points[points[:,2]!=0]


(10, 2)

In [11]:
def calculate_centroids(data, labels):
    unique_labels = np.unique(labels)
    centroids = []

    for label in unique_labels:
        cluster_points = data[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        centroids.append((label, centroid))
    return centroids


def find_closest_centroid(previous_centroids, current_centroids, id_mapping, next_id, frame_counts, frame_threshold=20):
    new_centroids = []
    for curr_label, curr_centroid in current_centroids:
        min_distance = float('inf')
        closest_centroid = None
        closest_prev_label = None

        for prev_label, prev_centroid in previous_centroids:
            distance = np.linalg.norm(curr_centroid - prev_centroid)
            if distance < min_distance:
                min_distance = distance
                closest_centroid = curr_centroid
                closest_prev_label = prev_label

        if closest_prev_label is not None and min_distance < 0.1:  # Threshold distance
            # Use existing ID if close to a previous centroid, initializing if necessary
            centroid_id = id_mapping.get(closest_prev_label, next_id)
            if closest_prev_label not in id_mapping:
                id_mapping[closest_prev_label] = centroid_id
                frame_counts[centroid_id] = 0  # Initialize frame count if first occurrence
            frame_counts[centroid_id] += 1  # Increment frame count
        else:
            # Assign a new ID if no close match found
            centroid_id = next_id
            id_mapping[curr_label] = centroid_id
            frame_counts[centroid_id] = 1  # Initialize frame count
            next_id += 1

        # Store the ID in the mapping and in the new_centroids list
        new_centroids.append((centroid_id, curr_centroid))

    # Filter centroids based on frame threshold
    valid_centroids = [(cid, c) for cid, c in new_centroids if frame_counts[cid] >= frame_threshold]
    return valid_centroids, next_id


In [30]:
previous_centroids = []
id_mapping = {}
frame_counts = {}
next_id = 0
cmap = cm.get_cmap("tab20", 20)
frames = 5
for index, row in mergedRadarData.iterrows():
    if index >= len(mergedRadarData) - frames:
        break

    # mergedRadarData['radar1_doppler']
    print('index:', index)
    fig = plt.figure(figsize=(12, 6))
    x_pcd = [e for elm in mergedRadarData['radar1_x_coord'][index:index + frames] for e in elm]
    y_pcd = [e for elm in mergedRadarData['radar1_y_coord'][index:index + frames] for e in elm]
    dop_pcd = [e for elm in mergedRadarData['radar1_dopplerIdx'][index:index + frames] for e in elm]
    
    points = np.concatenate((np.array(x_pcd).reshape(-1,1),np.array(y_pcd).reshape(-1,1), np.array(dop_pcd).reshape(-1,1)), axis=1)
    # points = points[points[:,2]!=0]
    img1 = plt.scatter(points[:,0], points[:,1], marker='o',s=4)

    clustering = DBSCAN(eps=0.001, min_samples=5).fit(points[:,:2])
    cluster_labels = np.array(clustering.labels_)
    current_centroids = calculate_centroids(points[:,:2], cluster_labels)

    valid_centroids, next_id = find_closest_centroid(previous_centroids, current_centroids, id_mapping, next_id, frame_counts, frame_threshold=10)
    previous_centroids = current_centroids  
    print('valid_centroids:', valid_centroids)

    for centroid_id, centroid in valid_centroids:
        color = cmap(centroid_id % 20)  
        plt.scatter(centroid[0], centroid[1], marker='o', s=100, color=color, label=f'Cluster ID {centroid_id}')


    plt.title('Radar_1 PCD')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(-10, 10)
    plt.ylim(0, 10)
    plt.legend()
    plt.tight_layout()
    # plt.show()
    plt.savefig(f"visualization/test_{str(index)}.png")
    plt.close()

    # if index==100:
    #     break
    
    

/tmp/ipykernel_1221912/2376270481.py:5: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("tab20", 20)
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


index: 0
valid_centroids: []
index: 1
valid_centroids: []


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


index: 2
valid_centroids: []
index: 3
valid_centroids: []


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


...

index: 219
valid_centroids: []
index: 220
valid_centroids: [(39, array([-0.47842824,  1.66551266]))]
index: 221
valid_centroids: [(39, array([-0.44632948,  1.74941573]))]
index: 222
valid_centroids: [(39, array([-0.71362712,  2.013511  ]))]


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


index: 223
valid_centroids: []
index: 224
valid_centroids: []


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


index: 225
valid_centroids: []
index: 226
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 227
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 228
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 229
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 230
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 231
valid_centroids: [(3, array([0.63806957, 0.457809  ]))]
index: 232
valid_centroids: [(40, array([-0.53520365,  2.52453334])), (3, array([0.63806957, 0.457809  ]))]
index: 233
valid_centroids: [(40, array([-0.50087779,  2.43507737])), (42, array([-0.079077  ,  0.03688737])), (3, array([0.63806957, 0.457809  ]))]
index: 234
valid_centroids: [(40, array([-0.42876469,  2.29573606])), (42, array([-0.079077  ,  0.03688737])), (3, array([0.63806957, 0.457809  ])), (43, array([1.40975201, 1.48907614])), (44, array([-2.45411372,  2.164325  ])), (45, array([-4.5591979 ,  4.81573629])), (46, array([-4.75280046,  5.46467781]))]